In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 12.3MB 838kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 21.6MB 111.4MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 18.2MB 170kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 6.3MB 2.6MB/s 
     |████████████████████████████████| 153kB 4.5MB/s 
     |████████████████████████████████| 235kB 8.5MB/s 
     |████████████████████████████████| 2.1MB 10.7MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.5.0-cp36-none-any.whl size=267918 sha256=758ba9e6485468da4ce550bfdeb98ca4c0701726d547efbe5863caf911586766
  Stored in directory: /root/.cache/pip/wheels/ec/51/31/5786f2ac419ee312f22d4d2877da05f20e7f2d430e22917daf
Su

In [3]:
import os.path as osp

import torch
import torch.utils.data as data_utils
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

from torch_geometric.utils import (negative_sampling, remove_self_loops,
                                   add_self_loops, convert)
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv  # noqa
from torch_geometric.utils import train_test_split_edges
import numpy as np
import pandas as pd
import networkx as nx
import scipy.sparse as sp
torch.manual_seed(12345)


In [4]:
g = nx.read_edgelist('/content/drive/My Drive/Dataset-Yeast/yeast.edgelist')
print("type of g =", type(g))
g_torch=convert.from_networkx(g)
print("type of g_torch =", type(g_torch))
data=g_torch

type of g = <class 'networkx.classes.graph.Graph'>
type of g_torch = <class 'torch_geometric.data.data.Data'>


In [8]:
g.remove_node("-")

In [9]:
len(g.nodes)

6525

In [10]:
print("type of g =", type(g))
g_torch=convert.from_networkx(g)
print("type of g_torch =", type(g_torch))
data=g_torch

type of g = <class 'networkx.classes.graph.Graph'>
type of g_torch = <class 'torch_geometric.data.data.Data'>


In [11]:
# Train/validation/test
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data, val_ratio=0.02, test_ratio=0.02 )


adj = nx.adjacency_matrix(g)
adj = sp.coo_matrix(adj)
adj_ = adj + sp.eye(adj.shape[0])
rowsum = np.array(adj_.sum(1))
degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()


In [12]:

adj_norm_np=adj_normalized.toarray()
# data.x=data_utils.TensorDataset(torch.from_numpy(adj_norm_np).float())

data.x=torch.tensor(adj_norm_np).float()


print("Type of data.x", type(data.x))
print("value of data.x", data.x)

print("shape of data.x", data.x.shape)


Type of data.x <class 'torch.Tensor'>
value of data.x tensor([[0.0020, 0.0023, 0.0015,  ..., 0.0000, 0.0000, 0.0000],
        [0.0023, 0.0105, 0.0033,  ..., 0.0000, 0.0000, 0.0000],
        [0.0015, 0.0033, 0.0042,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.5000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.5000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0526]])
shape of data.x torch.Size([6525, 6525])


In [18]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(6525, 32)
        self.conv2 = GCNConv(32, 16)

    def forward(self, pos_edge_index, neg_edge_index):

        x = F.relu(self.conv1(data.x, data.train_pos_edge_index))
        x = self.conv2(x, data.train_pos_edge_index)

        total_edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)
        x_j = torch.index_select(x, 0, total_edge_index[0])
        x_i = torch.index_select(x, 0, total_edge_index[1])
        return torch.einsum("ef,ef->e", x_i, x_j)



In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
model=model.float()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)


In [20]:

def get_link_labels(pos_edge_index, neg_edge_index):
    link_labels = torch.zeros(pos_edge_index.size(1) +
                              neg_edge_index.size(1)).float().to(device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels


def train():
    model.train()
    optimizer.zero_grad()

    x, pos_edge_index = data.x, data.train_pos_edge_index

    _edge_index, _ = remove_self_loops(pos_edge_index)
    pos_edge_index_with_self_loops, _ = add_self_loops(_edge_index,
                                                       num_nodes=x.size(0))

    neg_edge_index = negative_sampling(
        edge_index=pos_edge_index_with_self_loops, num_nodes=x.size(0),
        num_neg_samples=pos_edge_index.size(1))

    link_logits = model(pos_edge_index, neg_edge_index)
    link_labels = get_link_labels(pos_edge_index, neg_edge_index)

    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
    loss.backward()
    optimizer.step()

    return loss


def test():
    model.eval()
    perfs = []
    for prefix in ["val", "test"]:
        pos_edge_index, neg_edge_index = [
            index for _, index in data("{}_pos_edge_index".format(prefix),
                                       "{}_neg_edge_index".format(prefix))
        ]
        link_probs = torch.sigmoid(model(pos_edge_index, neg_edge_index))
        link_labels = get_link_labels(pos_edge_index, neg_edge_index)
        link_probs = link_probs.detach().cpu().numpy()
        link_labels = link_labels.detach().cpu().numpy()
        perfs.append(roc_auc_score(link_labels, link_probs))
    return perfs


In [21]:
import time

In [22]:
best_val_perf = test_perf = 0
for epoch in range(1, 20):
    t = time.time()
    train_loss = train()
    val_perf, tmp_test_perf = test()
    if val_perf > best_val_perf:
        best_val_perf = val_perf
        test_perf = tmp_test_perf
    log = 'Epoch: {:03d}, Loss: {:.4f}, Val: {:.4f}, Test: {:.4f}, time: {:.5f}'
    print(log.format(epoch, train_loss, best_val_perf, test_perf, time.time() - t ))


Epoch: 001, Loss: 0.6931, Val: 0.8767, Test: 0.8753, time: 1.32339
Epoch: 002, Loss: 0.6909, Val: 0.8779, Test: 0.8765, time: 1.17758
Epoch: 003, Loss: 0.6834, Val: 0.8784, Test: 0.8771, time: 1.19965
Epoch: 004, Loss: 0.6689, Val: 0.8787, Test: 0.8774, time: 1.17300
Epoch: 005, Loss: 0.6496, Val: 0.8790, Test: 0.8777, time: 1.19252
Epoch: 006, Loss: 0.6366, Val: 0.8793, Test: 0.8780, time: 1.17968
Epoch: 007, Loss: 0.6427, Val: 0.8796, Test: 0.8784, time: 1.18991
Epoch: 008, Loss: 0.6452, Val: 0.8799, Test: 0.8788, time: 1.17436
Epoch: 009, Loss: 0.6347, Val: 0.8803, Test: 0.8792, time: 1.17141
Epoch: 010, Loss: 0.6260, Val: 0.8806, Test: 0.8795, time: 1.16460
Epoch: 011, Loss: 0.6252, Val: 0.8808, Test: 0.8797, time: 1.16927
Epoch: 012, Loss: 0.6260, Val: 0.8809, Test: 0.8799, time: 1.17582
Epoch: 013, Loss: 0.6248, Val: 0.8810, Test: 0.8799, time: 1.16960
Epoch: 014, Loss: 0.6210, Val: 0.8810, Test: 0.8799, time: 1.18217
Epoch: 015, Loss: 0.6156, Val: 0.8810, Test: 0.8799, time: 1.1